In [1]:
from util import *

# 2015

## Day 1

In [2]:
stairs = lambda s: 1 if s == '(' else -1

def santa_stairs(steps): return sum(map(stairs, steps))

assert 0 == santa_stairs("()()") == santa_stairs("))((")
assert 3 == santa_stairs("(()(()(") == santa_stairs("(((") == santa_stairs("))(((((")
assert -1 == santa_stairs("())") == santa_stairs("))(")

input1 = cat(Input("1"))
assert 232 == santa_stairs(input1)

In [3]:
def when_enter_basement(steps):
    current_floor = 0
    for i, s in enumerate(steps):
        current_floor += stairs(s)
        if current_floor < 0: return i + 1

assert 1783 == when_enter_basement(input1)


## Day 2

In [4]:
def surface_area_with_slack(dimensions):
    l, w, h = sorted(dimensions)
    return 2*l*w + 2*w*h + 2*h*l + l*w

input2 = Input(2, line_parser=integers)
assert 1606483 == sum(map(surface_area_with_slack, input2))

In [5]:
def required_ribbon(dimensions):
    l, w, h = sorted(dimensions)
    return 2 * (l + w) + l * w * h
    
assert 34 == required_ribbon((2,3,4))
assert 14 == required_ribbon((1,1,10))
assert 3842356 == sum(map(required_ribbon, input2))


## Day 3

In [6]:
headings = {'>': right, '<': left, '^': up, 'v': down}

def unique_houses(directions):
    location, houses_received_presents = 0, {0}
    for d in directions:
        location += headings[d]
        houses_received_presents.add(location)
        
    return len(houses_received_presents)
        
assert 2 == unique_houses('>')
assert 4 == unique_houses('^>v<')
assert 2 == unique_houses('^v^v^v^v^v')

input3 = cat(Input(3))
assert 2572 == unique_houses(input3)

In [7]:
def unique_houses_robo_santa(directions):
    santa_location, robo_santa_location = complex(0,0), complex(0,0)
    presents = {santa_location}
    for s, r in zip(directions[0::2], directions[1::2]):
        santa_location += headings[s]
        robo_santa_location += headings[r]
        presents |= {santa_location, robo_santa_location}
    return len(presents)
        
assert 3 == unique_houses_robo_santa('^v')
assert 3 == unique_houses_robo_santa('^>v<')
assert 11 == unique_houses_robo_santa('^v^v^v^v^v')

input3 = cat(Input(3))
assert 2631 == unique_houses_robo_santa(input3)

## Day 4

In [8]:
from hashlib import md5

def crack_secret_key(key, zeros=5):
    i = 0
    while True:
        to_hash = key + str(i)
        hashed = md5(to_hash.encode()).hexdigest()
        if hashed[:zeros] == '0'*zeros:
            return i
        i += 1

assert 609043 == crack_secret_key('abcdef')
assert 1048970 == crack_secret_key('pqrstuv')

input4 = 'yzbqklnj'
assert 282749 == crack_secret_key(input4, zeros=5)
assert 9962624 == crack_secret_key(input4, zeros=6)

## Day 5

In [9]:
def is_nice_string(s):
    return any(a == b for a, b in zip(s[:-1], s[1:])) and \
           sum(1 for i in s if i in 'aeiou') >= 3 and \
           not any(x in s for x in {'ab', 'cd', 'pq', 'xy'})

assert True == is_nice_string('ugknbfddgicrmopn')
assert True == is_nice_string('aaa')
assert False == is_nice_string('jchzalrnumimnmhp')
assert False == is_nice_string('haegwjzuvuyypxyu')
assert False == is_nice_string('dvszwmarrgswjxmb')

input5 = Input(5)
assert 236 == sum(1 for i in input5 if is_nice_string(i))

In [10]:
def is_nice_string2(s):
    return any(s[i:i+2] in s[i+2:] for i in range(len(s)-3)) and \
           any(a == c for a,c in zip(s, s[2:]))
    
assert True == is_nice_string2('qjhvhtzxzqqjkmpb')
assert True == is_nice_string2('xxyxx')
assert False == is_nice_string2('uurcxstgmygtbstg')
assert False == is_nice_string2('ieodomkazucvgmuy')
assert False == is_nice_string2('ugknbfddgicrmopn')
assert False == is_nice_string2('aaa')
assert False == is_nice_string2('jchzalrnumimnmhp')
assert False == is_nice_string2('haegwjzuvuyypxyu')
assert False == is_nice_string2('dvszwmarrgswjxmb')

assert 51 == sum(1 for i in input5 if is_nice_string2(i))

## Day 6

In [11]:
def execute(instructions, actions, lights = None):
    lights = [[False]*1000 for _ in range(1000)] if lights is None else lights
    instructions = re.findall("(toggle|turn on|turn off)\s(\d*),(\d*)\sthrough\s(\d*),(\d*)", instructions)
    for (a, x0, y0, x1, y1) in instructions:
        coordinates = ((x, y) for x in rangei(int(x0), int(x1)) for y in rangei(int(y0), int(y1)))
        for x, y in coordinates: 
            lights[x][y] = actions[a](lights[x][y])
    return lights

fuel_needed = {
    'toggle': lambda x: 0 if x == 1 else 1, 
    'turn on': lambda _: 1, 
    'turn off': lambda _: 0, 
}

assert all(execute('turn on 0,0 through 999,999', fuel_needed))
assert all(execute('toggle 0,0 through 0,999', fuel_needed)[0])
assert all(i[0] for i in execute('toggle 0,0 through 999,0', fuel_needed))

input6 = cat(Input(6))
assert 377891 == sum(i for sublist in execute(input6, fuel_needed) for i in sublist)

In [12]:
part_2 = {
    'toggle': lambda x: x + 2, 
    'turn on': lambda x: x + 1, 
    'turn off': lambda x: max(x - 1, 0), 
}

assert 14110788 == sum(i for sublist in execute(input6, part_2) for i in sublist)

## Day 7

In [13]:
@lru_cache()
def get_value(key):
    try:
        return int(key)
    except ValueError:
        pass

    cmd = data[key]

    if "NOT" in cmd:      return ~get_value(cmd[1])
    elif "AND" in cmd:    return get_value(cmd[0]) & get_value(cmd[2])
    elif "OR" in cmd:     return get_value(cmd[0]) | get_value(cmd[2])
    elif "LSHIFT" in cmd: return get_value(cmd[0]) << get_value(cmd[2])
    elif "RSHIFT" in cmd: return get_value(cmd[0]) >> get_value(cmd[2])
    else:                 return get_value(cmd[0])

def generate_data(text):
    return {tokens[-1] : tokens[:-2] for tokens in map(str.split, text)}

data = generate_data(Input(7))

assert 16076 == get_value("a")

In [14]:
data["b"] = str(get_value("a"))
get_value.cache_clear()
assert 32790 == get_value("a")

## Day 8

In [ ]:
input8 = Input(8)

assert 1333 == sum(len(line) - len(eval(line)) for line in input8)

Need more work on this one...

In [ ]:
sum(len("\"{}\"".format(re.escape(line))) - len(line) for line in "\"abc\"")

## Day 9

In [ ]:
test = """London to Dublin = 464\nLondon to Belfast = 518\nDublin to Belfast = 141"""

def parse(text):
    distances, cities = dict(), set()
    for (origin, destination, distance) in re.findall("(\w+)\sto\s(\w+)\s=\s(\d*)", text):
        distances[fs((origin, destination))] = int(distance)
        cities |= {origin, destination}
    return distances, cities
    
def find_dist(distances, cities, cmp=min):
    total = []
    for p in permutations(cities):
        total.append(sum(distances[fs((c1, c2))] for (c1, c2) in zip(p[:-1], p[1:])))
    return cmp(total)

input9 = cat(Input(9))
assert 207 == find_dist(*parse(input9), cmp=min)

In [ ]:
assert 804 == find_dist(*parse(input9), cmp=max)

## Day 10

In [ ]:
repeat_and_say = lambda x: ''.join(str(sum(1 for _ in k)) + i for i, k in groupby(x))

assert 252594 == len(repeat(repeat_and_say, 40, '1113222113'))

In [ ]:
assert 3579328 == len(repeat(repeat_and_say, 50, '1113222113'))
